In [7]:
import csv
import datetime
from operator import sub
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing, ensemble
import sys


In [2]:
feature_cols = ["employee_index","country_residence","sex","age","first_join_date","new_customer_index","seniority","primary","customer_type","customer_relation","residence_idx","foreigner_idx","channel","deceased","address","province_code","province_name","activity_idx","gross_income","segment"]
target_cols=["saving_account","guarantees","current_account","derivada_account","payroll_account","junior_account","mas_particular_account","particular_account","particular_plus_account","short-term_deposits","medium-term_deposits","long-term_deposits","e-account","funds","mortgage","pensions","loans","taxes","credit_card","securities","home_account","payroll","pensions_1","direct_debit"]
target_cols = target_cols[2:]

all_cols = feature_cols+target_cols


In [3]:
def getTarget(row):
    tlist = []
    for col in target_cols:
            if row[col].strip() in ['', 'NA']:
                    target = 0
            else:
                    target = int(float(row[col]))
            tlist.append(target)
    return tlist


In [4]:
def processData(in_file_name, cust_dict):
    x_vars_list = []
    y_vars_list = []
    for row in csv.DictReader(in_file_name, fieldnames = ( "date","id","employee_index","country_residence","sex",
                                                          "age","first_join_date","new_customer_index","seniority",
                                                          "primary","last_date_primary","customer_type","customer_relation",
                                                          "residence_idx","foreigner_idx","spouse_idx","channel","deceased",
                                                          "address","province_code","province_name","activity_idx",
                                                          "gross_income","segment","saving_account","guarantees",
                                                          "current_account","derivada_account","payroll_account",
                                                          "junior_account","mas_particular_account","particular_account",
                                                          "particular_plus_account","short-term_deposits","medium-term_deposits",
                                                          "long-term_deposits","e-account","funds","mortgage","pensions","loans",
                                                          "taxes","credit_card","securities","home_account","payroll",
                                                          "pensions_1","direct_debit" )):
            # use only the four months as specified by breakfastpirate #
            if row['date'] not in ['2015-05-28', '2015-06-28', '2016-05-28', '2016-06-28']:
                    continue

            cust_id = int(row['id'])
            if row['date'] in ['2015-05-28', '2016-05-28']:
                    target_list = getTarget(row)
                    cust_dict[cust_id] =  target_list[:]
                    continue

            x_vars = []

            for col in feature_cols:
                val = row[col].strip()
                x_vars.append(val)
            #for col in cat_cols:
            #       x_vars.append( getIndex(row, col) )
            #x_vars.append( getAge(row) )
            #x_vars.append( getCustSeniority(row) )
            #x_vars.append( getRent(row) )
            #x_vars = row
            if row['date'] == '2016-06-28':
                    prev_target_list = cust_dict.get(cust_id, [0]*22)
                    x_vars_list.append(x_vars + prev_target_list)
            elif row['date'] == '2015-06-28':
                    prev_target_list = cust_dict.get(cust_id, [0]*22)
                    target_list = getTarget(row)
                    new_products = [max(x1 - x2,0) for (x1, x2) in zip(target_list, prev_target_list)]
                    if sum(new_products) > 0:
                            for ind, prod in enumerate(new_products):
                                    if prod>0:
                                            assert len(prev_target_list) == 22
                                            x_vars_list.append(x_vars+prev_target_list)
                                            y_vars_list.append(ind)

    return x_vars_list, y_vars_list, cust_dict


In [5]:
def runXGB(train_X, train_y, seed_val=0):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.05
    param['max_depth'] = 8
    param['silent'] = 1
    param['num_class'] = 22
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = 50

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)
    model = xgb.train(plst, xgtrain, num_rounds)
    return model


In [16]:
if __name__ == "__main__":
    start_time = datetime.datetime.now()
    data_path = "./input/"
    print(len(all_cols))
    #train_file =  open(data_path + "train_ver2.csv")
    train_file =  open(data_path + "train_20percent.csv")
    x_vars_list, y_vars_list, cust_dict = processData(train_file, {})
    print(len(x_vars_list[1]))
    #print(len(x_vars_list))
    #train_X = np.array(x_vars_list)
    #train_y = np.array(y_vars_list)

    df = pd.DataFrame.from_records(x_vars_list, columns=all_cols)
    print(len(df))
    
    df['y']=y_vars_list
    print(df.columns)
    df['age']   = pd.to_numeric(df['age'], errors='coerce')
    df['seniority']   = pd.to_numeric(df['seniority'], errors='coerce')
    ##df = df[df['seniority']>0]
    ##df.dropna(axis=0)
    #df.dropna()
    print(len(df))
    condition=~pd.isnull(df).any(axis=1)
    df=df[condition]
#     for ind, feature_col in enumerate(feature_cols):
#         uniq_values = list(df[feature_col].unique())
#         print("WEICHUN checked:",feature_col,len(uniq_values))
#         for i in range(len(uniq_values)):
#             print(feature_col,uniq_values[i])
    #print(len(x_vars_list),len(y_vars_list))
    #print(train_X.shape,train_y.shape)

    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    feature_category_cols = feature_cols
    for ind, feature_col in enumerate(feature_category_cols):
        le.fit(df[feature_col])
        df[feature_col]=le.transform(df[feature_col])
    print("weichun exiting")


42
42
182068
Index([u'employee_index', u'country_residence', u'sex', u'age',
       u'first_join_date', u'new_customer_index', u'seniority', u'primary',
       u'customer_type', u'customer_relation', u'residence_idx',
       u'foreigner_idx', u'channel', u'deceased', u'address', u'province_code',
       u'province_name', u'activity_idx', u'gross_income', u'segment',
       u'current_account', u'derivada_account', u'payroll_account',
       u'junior_account', u'mas_particular_account', u'particular_account',
       u'particular_plus_account', u'short-term_deposits',
       u'medium-term_deposits', u'long-term_deposits', u'e-account', u'funds',
       u'mortgage', u'pensions', u'loans', u'taxes', u'credit_card',
       u'securities', u'home_account', u'payroll', u'pensions_1',
       u'direct_debit', u'y'],
      dtype='object')
182068
weichun exiting


In [17]:
print(len(df))

181792


In [18]:
df_y = df['y']

In [20]:
len(df_y)

181792

In [16]:
#     sys.exit()
print(np.unique(train_y))
# del x_vars_list, y_vars_list
# train_file.close()
# print(train_X.shape, train_y.shape)
# print(datetime.datetime.now()-start_time)
test_file = open(data_path + "test_ver2.csv")


x_vars_list, y_vars_list, cust_dict = processData(test_file, cust_dict)
test_X = np.array(x_vars_list)
print("wei")
print x_vars_list
sys.exit()
del x_vars_list
test_file.close()
print(test_X.shape)
print(datetime.datetime.now()-start_time)

print("Building model..")
model = runXGB(train_X, train_y, seed_val=0)
del train_X, train_y
print("Predicting..")
xgtest = xgb.DMatrix(test_X)
preds = model.predict(xgtest)
del test_X, xgtest
print(datetime.datetime.now()-start_time)

print("Getting the top products..")
target_cols = np.array(target_cols)
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)[:,:7]
test_id = np.array(pd.read_csv(data_path + "test_ver2.csv", usecols=['ncodpers'])['ncodpers'])
final_preds = [" ".join(list(target_cols[pred])) for pred in preds]



[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


In [ ]:
x_vars_list

In [ ]:
1453*0.8